
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# BERT を事前トレーニングするためのデータセット

:label: `sec_bert-dataset`

 :numref: `sec_bert`で実装されているように BERT モデルを事前トレーニングするには、マスクされた言語モデリングと次の文の予測という 2 つの事前トレーニング タスクを容易にするために、理想的な形式でデータセットを生成する必要があります。一方で、元の BERT モデルは 2 つの巨大なコーパス BookCorpus と英語版 Wikipedia ( :numref: `subsec_bert_pretraining_tasks`を参照) を連結したもので事前トレーニングされているため、この本のほとんどの読者にとって実行するのは困難です。一方で、既製の事前トレーニング済み BERT モデルは、医療などの特定の分野のアプリケーションには適合しない可能性があります。したがって、カスタマイズされたデータセットで BERT を事前トレーニングすることが一般的になりつつあります。 BERT 事前トレーニングのデモンストレーションを容易にするために、より小さなコーパス WikiText-2 :cite: `Merity.Xiong.Bradbury.ea.2016`を使用します。

 :numref: `sec_word2vec_data`で word2vec の事前トレーニングに使用された PTB データセットと比較すると、WikiText-2 (i) は元の句読点を保持しているため、次の文の予測に適しています。 (ii) 元のケースと番号を保持します。 (iii) 2 倍以上大きい。


In [1]:
import os
import random
import torch
from d2l import torch as d2l


[ **WikiText-2 データセット**] では、各行は句読点とその前のトークンの間にスペースが挿入された段落を表します。少なくとも 2 つの文を含む段落は保持されます。文を分割するには、簡単にするために区切り文字としてピリオドのみを使用します。より複雑な文分割テクニックについては、このセクションの最後にある演習で説明します。


In [2]:
#@save
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')

#@save
def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r') as f:
        lines = f.readlines()
    # Uppercase letters are converted to lowercase ones
    paragraphs = [line.strip().lower().split(' . ')
                  for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs


## 事前トレーニングタスク用のヘルパー関数の定義

以下では、次の文の予測とマスクされた言語モデリングという 2 つの BERT 事前トレーニング タスクのヘルパー関数を実装することから始めます。これらのヘルパー関数は、後で生のテキスト コーパスを理想的な形式のデータセットに変換して BERT を事前学習するときに呼び出されます。

###  [**次文予測タスクの生成**]

 :numref: `subsec_nsp`の説明に従って、 `_get_next_sentence`関数は二値分類タスクのトレーニング サンプルを生成します。


In [3]:
#@save
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # `paragraphs` is a list of lists of lists
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next


次の関数は、 `_get_next_sentence`関数を呼び出して、入力`paragraph`から次の文を予測するためのトレーニング例を生成します。ここで、 `paragraph`文のリストであり、各文はトークンのリストです。引数`max_len`事前トレーニング中の BERT 入力シーケンスの最大長を指定します。


In [4]:
#@save
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(
            paragraph[i], paragraph[i + 1], paragraphs)
        # Consider 1 '<cls>' token and 2 '<sep>' tokens
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph


### [**マスクされた言語モデリング タスクの生成**]

 :label: `subsec_prepare_mlm_data`

 BERT 入力シーケンスからマスクされた言語モデリング タスクのトレーニング サンプルを生成するために、次の`_replace_mlm_tokens`関数を定義します。その入力では、 `tokens` BERT 入力シーケンスを表すトークンのリスト、 `candidate_pred_positions`は特別なトークンを除く BERT 入力シーケンスのトークン インデックスのリスト (特殊なトークンはマスクされた言語モデリング タスクでは予測されません)、 `num_mlm_preds`次のことを示します。予測の数 (予測する 15% のランダム トークンを思い出してください)。 :numref: `subsec_mlm`のマスクされた言語モデリング タスクの定義に従って、各予測位置で、入力が特別な「&amp;lt;mask&amp;gt;」に置き換えられる場合があります。トークンまたはランダムなトークン、または変更されないままになります。最終的に、この関数は、置換後の入力トークン、予測が行われるトークン インデックス、およびこれらの予測のラベルを返します。


In [5]:
#@save
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # For the input of a masked language model, make a new copy of tokens and
    # replace some of them by '<mask>' or random tokens
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # Shuffle for getting 15% random tokens for prediction in the masked
    # language modeling task
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80% of the time: replace the word with the '<mask>' token
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10% of the time: keep the word unchanged
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10% of the time: replace the word with a random word
            else:
                masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels


前述の`_replace_mlm_tokens`関数を呼び出すことにより、次の関数は BERT 入力シーケンス ( `tokens` ) を入力として受け取り、入力トークンのインデックス ( :numref: `subsec_mlm`で説明されているように可能なトークン置換後)、予測が行われるトークン インデックスを返します。これらの予測のラベル インデックス。


In [6]:
#@save
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    # `tokens` is a list of strings
    for i, token in enumerate(tokens):
        # Special tokens are not predicted in the masked language modeling
        # task
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 15% of random tokens are predicted in the masked language modeling task
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(
        tokens, candidate_pred_positions, num_mlm_preds, vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels,
                                       key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]


## テキストを事前トレーニング データセットに変換する

これで、BERT を事前トレーニングするために`Dataset`クラスをカスタマイズする準備がほぼ整いました。その前に、ヘルパー関数`_pad_bert_inputs`定義して [**特別な「&amp;lt;pad&amp;gt;」を追加する]**必要があります。**入力へのトークン。** ] その引数の`examples` 2 つの事前トレーニング タスクのヘルパー関数`_get_nsp_data_from_paragraph`および`_get_mlm_data_from_tokens`からの出力が含まれています。


In [7]:
#@save
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens,  = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for (token_ids, pred_positions, mlm_pred_label_ids, segments,
         is_next) in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (
            max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
            max_len - len(segments)), dtype=torch.long))
        # `valid_lens` excludes count of '<pad>' tokens
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(torch.tensor(pred_positions + [0] * (
            max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # Predictions of padded tokens will be filtered out in the loss via
        # multiplication of 0 weights
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (
                max_num_mlm_preds - len(pred_positions)),
                dtype=torch.float32))
        all_mlm_labels.append(torch.tensor(mlm_pred_label_ids + [0] * (
            max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return (all_token_ids, all_segments, valid_lens, all_pred_positions,
            all_mlm_weights, all_mlm_labels, nsp_labels)


2 つの事前トレーニング タスクのトレーニング例を生成するためのヘルパー関数と、入力をパディングするためのヘルパー関数を組み合わせて、次の`_WikiTextDataset`クラスを [ **BERT 事前トレーニング用の WikiText-2 データセット**] としてカスタマイズします。 `__getitem__`関数を実装すると、WikiText-2 コーパスの 1 対の文から生成された事前トレーニング (マスクされた言語モデリングと次の文の予測) サンプルに任意にアクセスできます。

元の BERT モデルは、語彙サイズが 30000 の WordPiece 埋め込みを使用します (cite: `Wu.Schuster.Chen.ea.2016` )。 WordPiece のトークン化方法は、 :numref: `subsec_Byte_Pair_Encoding`の元のバイト ペア エンコード アルゴリズムをわずかに変更したものです。簡単にするために、トークン化には`d2l.tokenize`関数を使用します。出現回数が 5 回未満の頻度の低いトークンはフィルターで除外されます。


In [8]:
#@save
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # Input `paragraphs[i]` is a list of sentence strings representing a
        # paragraph; while output `paragraphs[i]` is a list of sentences
        # representing a paragraph, where each sentence is a list of tokens
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]
        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # Get data for the next sentence prediction task
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(
                paragraph, paragraphs, self.vocab, max_len))
        # Get data for the masked language model task
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab)
                      + (segments, is_next))
                     for tokens, segments, is_next in examples]
        # Pad inputs
        (self.all_token_ids, self.all_segments, self.valid_lens,
         self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(
            examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx],
                self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)


`_read_wiki`関数と`_WikiTextDataset`クラスを使用して、[ **WikiText-2 データセットをダウンロードし、そこから事前トレーニング サンプルを生成する**] 次の`load_data_wiki`を定義します。


In [9]:
#@save
def load_data_wiki(batch_size, max_len):
    """Load the WikiText-2 dataset."""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                        shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab


バッチ サイズを 512 に設定し、BERT 入力シーケンスの最大長を 64 に設定して、[ **BERT 事前トレーニング サンプルのミニバッチの形状を出力**] します。各 BERT 入力シーケンスでは、マスクされた言語モデリング タスクに対して $10$ ($64 \times 0.15$) の位置が予測されることに注意してください。


In [10]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
     mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape,
          pred_positions_X.shape, mlm_weights_X.shape, mlm_Y.shape,
          nsp_y.shape)
    break

torch.Size([512, 64]) torch.Size([512, 64]) torch.Size([512]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512, 10]) torch.Size([512])



最後に語彙サイズを見てみましょう。頻度の低いトークンを除外した後でも、PTB データセットよりも 2 倍以上大きくなります。


In [11]:
len(vocab)

20256


## まとめ
- PTB データセットと比較すると、WikiText-2 データセットは元の句読点、大文字と小文字、数字が保持されており、2 倍以上大きくなっています。
-  WikiText-2 コーパスの 1 対の文から生成された事前トレーニング (マスクされた言語モデリングと次の文の予測) の例に任意にアクセスできます。

## 演習
1. 簡単にするために、文を分割するための唯一の区切り文字としてピリオドが使用されます。 spaCy や NLTK など、他の文分割テクニックを試してください。 NLTKを例に挙げます。最初に NLTK をインストールする必要があります: `pip install nltk` 。コードでは、まず`import nltk` 。次に、Punkt 文トークナイザーをダウンロードします: `nltk.download(&#39;punkt&#39;)` 。文章を分割するには、 `sentences = &#39;This is great ! Why not ?&#39;` 、 `nltk.tokenize.sent_tokenize(sentences)`を呼び出すと`[&#39;This is great !&#39;, &#39;Why not ?&#39;]` 2 つの文文字列のリストが返されます。
1. 頻度の低いトークンを除外しない場合の語彙サイズはどれくらいでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/1496)
